In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sign-language-mnist/american_sign_language.PNG
/kaggle/input/sign-language-mnist/amer_sign3.png
/kaggle/input/sign-language-mnist/sign_mnist_train.csv
/kaggle/input/sign-language-mnist/sign_mnist_test.csv
/kaggle/input/sign-language-mnist/amer_sign2.png
/kaggle/input/sign-language-mnist/sign_mnist_test/sign_mnist_test.csv
/kaggle/input/sign-language-mnist/sign_mnist_train/sign_mnist_train.csv


In [8]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout
from keras.optimizers import Adam

Using TensorFlow backend.


In [4]:
train = pd.read_csv('../input/sign-language-mnist/sign_mnist_train/sign_mnist_train.csv')
test= pd.read_csv('../input/sign-language-mnist/sign_mnist_test/sign_mnist_test.csv')

In [5]:
train.shape, train.head(), test.shape, test.head()

((27455, 785),
    label  pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  pixel8  \
 0      3     107     118     127     134     139     143     146     150   
 1      6     155     157     156     156     156     157     156     158   
 2      2     187     188     188     187     187     186     187     188   
 3      2     211     211     212     212     211     210     211     210   
 4     13     164     167     170     172     176     179     180     184   
 
    pixel9  ...  pixel775  pixel776  pixel777  pixel778  pixel779  pixel780  \
 0     153  ...       207       207       207       207       206       206   
 1     158  ...        69       149       128        87        94       163   
 2     187  ...       202       201       200       199       198       199   
 3     210  ...       235       234       233       231       230       226   
 4     185  ...        92       105       105       108       133       163   
 
    pixel781  pixel782  pixel783  pixel784  


In [6]:
X = train.iloc[:,1:].values
y = train.iloc[:,0].values

In [7]:
X.shape,X,y.shape,y

((27455, 784),
 array([[107, 118, 127, ..., 204, 203, 202],
        [155, 157, 156, ..., 103, 135, 149],
        [187, 188, 188, ..., 195, 194, 195],
        ...,
        [174, 174, 174, ..., 202, 200, 200],
        [177, 181, 184, ...,  64,  87,  93],
        [179, 180, 180, ..., 205, 209, 215]]),
 (27455,),
 array([ 3,  6,  2, ..., 18, 17, 23]))

In [9]:
X=X/255

In [16]:
train_X, test_X = X[:int(len(X)*0.75),:],X[int(len(X)*0.75):,:]
train_y, test_y =y[:int(len(X)*0.75)],y[int(len(X)*0.75):]

In [20]:
train_X.shape, test_X.shape, train_y.shape, test_y.shape

((20591, 28, 28, 1), (6864, 28, 28, 1), (20591,), (6864,))

In [19]:
train_X= train_X.reshape(train_X.shape[0],28,28,1)
test_X= test_X.reshape(test_X.shape[0],28,28,1)

In [21]:
np.unique(y)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24])

In [22]:
train_y= keras.utils.to_categorical(train_y,num_classes=25)
test_y= keras.utils.to_categorical(test_y,num_classes=25)

In [23]:
model = Sequential()

model.add(Conv2D(64,(3,3),activation='relu', input_shape=(28,28,1)))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.25))

model.add(Conv2D(128,(3,3),activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(128,activation='relu'))
model.add(Dense(84,activation='relu'))

model.add(Dense(25,activation='softmax'))


In [25]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [26]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 64)        640       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 13, 13, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 128)         0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 5, 5, 128)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 3200)             

In [27]:
class myCallbacks(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('loss')<0.05):
            print('accuracy is more then 99%')
            self.model.stop_training=True
            
callbacks = myCallbacks()

In [29]:
model.fit(train_X, train_y, epochs=50,batch_size=32, verbose=1, validation_data=(test_X,test_y),callbacks=[callbacks])

Train on 20591 samples, validate on 6864 samples
Epoch 1/50
20591/20591 [==============================] - 28s 1ms/step - loss: 0.2210 - accuracy: 0.9301 - val_loss: 0.0575 - val_accuracy: 0.9863
Epoch 2/50
20591/20591 [==============================] - 26s 1ms/step - loss: 0.0622 - accuracy: 0.9825 - val_loss: 0.0156 - val_accuracy: 0.9969
Epoch 3/50
20591/20591 [==============================] - 26s 1ms/step - loss: 0.0317 - accuracy: 0.9916 - val_loss: 0.0087 - val_accuracy: 0.9990
accuracy is more then 99%


In [30]:
validate_X=test.iloc[:,1:].values

In [31]:
validate_X=validate_X/255

In [32]:
validate_X= validate_X.reshape(validate_X.shape[0],28,28,1)

In [33]:
predicted = model.predict_classes(validate_X)

In [34]:
true_y= test.iloc[:,0].values

In [36]:
from sklearn.metrics import accuracy_score

acc = accuracy_score(true_y, predicted)
print(acc)

0.908393753485778


In [37]:
score = model.evaluate(test_X,test_y)
score[1]

ValueError: Error when checking target: expected dense_3 to have shape (25,) but got array with shape (1,)